In [1]:
import os
import re
import cv2
import sys
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
from pyzbar.pyzbar import *

In [2]:
# <QR Format>
# 식당: NWTAVR-MA-202112-식당이름
# 직원: NWTAVR-MS-202112-4077-01

QR_HEADER = "NWTAVR"				# 노원구청 교통행정과 등록팀 (내 QR인지 식별자 역할)
QR_CLASS_MA = "MA"					# 가맹점 = Member of Affiliation
QR_CLASS_MS = "MS"					# 직원 = Member of Staff
QR_YEAR = "2021"					# 대상년도 =========> 실행시 반드시 고칠 것
QR_MONTH = "12"						# 대상월   =========> 실행시 반드시 고칠 것
QR_YYYYMM = QR_YEAR + QR_MONTH		# "202112"
QR_SN = ["0"+str(i) for i in range(1,10)] + list(map(str, range(10, 21)))    # ["01", "02", ... "19", "20"]

affil_list = ["노란코끼리", "어장촌생선구이", "횡성목장", "항도(港都)", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "구내매점"]

code_list =  ["4080",    "4076",    "4079",    "4069",    "4078",    "4071",    "4068",    "4075",    "4077"   ]
name_list =  ["전현호",  "정성욱",  "박선녕",  "신상용",  "이정선",  "이희영",  "최미연",  "이재영",  "김재형" ]
color_list = ["#E32636", "#FFA500", "#FF5511", "#0FF1C3", "#AB3ED8", "#AAEE00", "#1155FF", "#FCE205", "#119617"]

name_by_code = {code_list[i]:name_list[i] for i in range(len(code_list))}		# {'4080':'전현호','4076':'정성욱',...,'4077':'김재형'}
color_by_code = {code_list[j]:color_list[j] for j in range(len(code_list))}		# {'4080':'#E32636','4076': '#FFA500',...,'4077':'#119617'}

MA_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MA + "-" + QR_YYYYMM + "-" + ".+")	# NWTAVR-MA-202112-노란코끼리
MS_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MS + "-" + QR_YYYYMM + "-" + ".+")	# NWTAVR-MS-202112-4077-01

In [3]:
# <장부 QR 동영상 읽기>
# 1) 가맹점별 동영상 1개 저장
# 2) 동영상 1개당 쓰레드 1개 실행

HOME_PATH = "C:/QRticket/"
MOV_PATH = HOME_PATH + QR_YYYYMM + "/"

file_list = os.listdir(MOV_PATH)		# ['0.MOV', '1.MOV', ... '10.MOV', '11.MOV']
num_of_file = len(file_list)			# 동영상 파일 개수
NUM_OF_THREAD = num_of_file				# = 쓰레드 개수

cap = [cv2.VideoCapture(os.path.join(MOV_PATH, file_list[i])) for i in range(num_of_file)]    # 동영상 파일 입력 ==> cap 리스트에 저장

In [4]:
# 파일 : 파일을 읽을 수 없으면 False
for i in range(num_of_file):
	print(f"#{i+1:>2}/{num_of_file} cap[{i:>2}] file ready: ", cap[i].isOpened())

# 1/12 cap[ 0] file ready:  True
# 2/12 cap[ 1] file ready:  True
# 3/12 cap[ 2] file ready:  True
# 4/12 cap[ 3] file ready:  True
# 5/12 cap[ 4] file ready:  True
# 6/12 cap[ 5] file ready:  True
# 7/12 cap[ 6] file ready:  True
# 8/12 cap[ 7] file ready:  True
# 9/12 cap[ 8] file ready:  True
#10/12 cap[ 9] file ready:  True
#11/12 cap[10] file ready:  True
#12/12 cap[11] file ready:  True


In [ ]:
# MOV 파일 하나씩 QR검출 돌릴 쓰레드
class QRDetectThread(Thread):
	
	# 초기화
	def __init__(self, cap=None, threadID=None):
		super(QRDetectThread, self).__init__(daemon=True)
		
		self.cap = cap												# 입력 MOV 파일
		self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))			# 이번 MOV 파일의 width
		self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))		# 이번 MOV 파일의 height
		self.frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))	# 이번 MOV 파일의 frame수
		self.fps = cap.get(cv2.CAP_PROP_FPS)						# 이번 MOV 파일의 frame rate
		
		self.detected_MA_set = set()    # 식당QR (중복없음)
		self.detected_MS_set = set()    # 식권QR (중복없음)
		
		
		self.fourcc = cv2.VideoWriter_fourcc(*'DIVX')
		self.out = cv2.VideoWriter(f"{MOV_PATH} + QR_result_{threadID}.avi", fourcc, fps, (width, height))
	
	# 쓰레드 실행
	def run(self):
		
		for frame_index in tqdm(range(self.frame_count)):

			# 읽어오지 못한 경우 이 시점에서 success = False ==> 쓰레드 종료
			success, frame = self.cap.read()
			if success == False:
				break

			# frame 1개에서 QR 찾기
			QRs_in_frame = decode(frame, symbols=[ZBarSymbol.QRCODE])
			num_of_QR = len(QRs_in_frame)
			# print(f"[전처리 전] 발견된 QR개수: {num_of_QR}")


			# 만약 하나도 발견되지 않았다면 다음 frame
			if num_of_QR == 0:
				# print("QR code not detected")
				continue


			# frame 1개에서 발견한 여러개 QR 중 eachQR은 1개
			for eachQR in QRs_in_frame:

				# QR 형식검증 : MS/MA패턴
				eachQR_data = eachQR.data.decode('utf-8')
				match_result_MA = MA_pattern.match(eachQR_data)
				match_result_MS = MS_pattern.match(eachQR_data)
				
				# QR이 가맹점도 직원도 아닌 경우 ==> 삭제
				# QR이 직원이면 저장, 가맹점이면 저장
				if match_result_MA is None:
					if match_result_MS is None:
						result.remove(eachQR)
						continue
					else:	# 직원인 경우
						self.detected_MS_set.add(match_result_MS.group())
						_header, _ms, _yyyymm, code, _sn = eachQR.data.decode('utf-8').split('-')
						color_hex_selected = color_by_code[code]
						color_selected = tuple(int(color_hex_selected[i:i+2], 16) for i in (5, 3, 1))	# HEX 문자열 -> BGR tuple값
						
				else:		# 가맹점인 경우
					self.detected_MA_set.add(match_result_MA.group())
					color_selected = (0, 0, 255)    # 식당QR은 빨간색

				
				# QR 테두리 선그리기 : QR 1개당 polygon 점의 개수(보통 4각형=4개)
				points = len(eachQR.polygon)

				# 경계선 컬러 설정 ==> 윗부분에 통합시킴
				# if match_result_MS is not None:
				# 	_header, _ms, _yyyymm, code, _sn = eachQR.data.decode('utf-8').split('-')
				# 	color_hex_selected = color_by_code[code]
				# 	color_selected = tuple(int(color_hex_selected[i:i+2], 16) for i in (5, 3, 1))	# HEX 문자열 -> BGR tuple값
				# else:
				# 	# _header, _class, _yyyymm, affil_name = eachQR.data.decode('utf-8').split('-')
				# 	color_selected = (0, 0, 255)    # 식당QR은 빨간색

				# QR 1개당 경계선 그리기
				for point in range(points):
					next_point = (point+1) % points
					cv2.line(frame, tuple(eachQR.polygon[point]), tuple(eachQR.polygon[next_point]), color_selected, 5)    # (B,G,R), 굵기

				# ========== [안쪽 for loop 내부] frame 1개 중 QR 1개 처리 끝 ==========

			# ========== [안쪽 for loop 종료] frame 1개 중 QR 전체 처리 끝 ==========
			# num_of_QR_preprocessed = len(QRs_in_frame)
			# print("Frame  #{:3}/{:3} 처리 끝    ====>    전처리 전: {:2} \t 전처리 후: {:2}".format(frame_index+1, frame_count, num_of_QR, num_of_QR_preprocessed))
			
			# frame 1개 처리 끝 ==> 저장
			self.out.write(frame)

		# ========== [바깥 for loop 종료] MOV 파일의 모든 frame 처리 끝 ==========
		
		# ========== 검출된 QR자료 정리 시작 ==========
		MA_list = list(self.detected_MA_set)
		MA_list = sorted(MA_list)
		MS_list = list(self.detected_MS_set)
		MS_list = sorted(MS_list)
		
		staff_list = []    # 이름 리스트
		count_list = []    # 개수 리스트
		
		if len(MA_list) != 1:
			print("[오류] QR 검출된 가맹점이 없거나, 2개 이상입니다.")
			sys.exit(0)
			
		if len(MS_list) != 0:
			_header, _ms, _yyyymm, current_code, _sn = MS_list[0].split("-")
			current_count = 0

			for index, eachMS in enumerate(MS_list):
				_header, _ms, _yyyymm, code, _sn = eachMS.split("-")

				if code != current_code:
					staff_list.append(current_code)
					count_list.append(current_count)

					current_code = code
					current_count = 1
				else:
					current_count += 1

				if (index == (len(MS_list) - 1)):
					staff_list.append(current_code)
					count_list.append(current_count)
					break
		else:
			print("[오류] QR 검출된 직원이 없습니다.")	
		
		self.cap.release()
		self.out.release()


In [204]:
# QR 검출된 직원 리스트



		if len(MS_list) != 0:
			_header, _ms, _yyyymm, current_code, _sn = MS_list[0].split("-")
			current_count = 0

			for index, eachMS in enumerate(MS_list):
				_header, _ms, _yyyymm, code, _sn = eachMS.split("-")

				if code != current_code:
					staff_list.append(current_code)
					count_list.append(current_count)

					current_code = code
					current_count = 1
				else:
					current_count += 1

				if (index == (len(MS_list) - 1)):
					staff_list.append(current_code)
					count_list.append(current_count)
					break
		else:
			print("QR 검출된 직원이 없습니다.")


In [205]:
print(staff_list)
print(count_list)

['4080']
[1]


In [206]:
# python 3.7부터 dict = ordered
final_MS_dict = {}

# 전체 code_list : ['4080', '4076', '4079', '4069', '4078', '4071', '4068', '4075', '4077']
for _code in code_list:
	name = name_by_code[_code]
	
	if _code not in staff_list:
		final_MS_dict[name] = 0
	else:
		final_MS_dict[name] = count_list[staff_list.index(_code)]

final_MS_dict

{'전현호': 1,
 '정성욱': 0,
 '박선녕': 0,
 '신상용': 0,
 '이정선': 0,
 '이희영': 0,
 '최미연': 0,
 '이재영': 0,
 '김재형': 0}

In [207]:
# 텍스트로 결과 보기

final_date = MA_list[0].split("-")[2]
final_affil = MA_list[0].split("-")[3]

print(f"{final_date[:4]}.{final_date[4:]}.  {final_affil}  총{sum(final_MS_dict.values())}개")
print("="*32)

for key, val in zip(final_MS_dict.keys(), final_MS_dict.values()):
	print(f"{key} : {val:2}개")

2021.12.  새싹비빔밥  총1개
전현호 :  1개
정성욱 :  0개
박선녕 :  0개
신상용 :  0개
이정선 :  0개
이희영 :  0개
최미연 :  0개
이재영 :  0개
김재형 :  0개


In [208]:
if NUM == 1:
	final_MA_list = []
	final_MS_list = []

final_MA_list.append(final_affil)
final_MS_list.append(final_MS_dict)

In [209]:
print(final_MA_list)
print(final_MS_list)

['노란코끼리', '어장촌생선구이', '횡성목장', '항도(港都)', '전주콩나루', '옛날칼국수', '칠리사이공', '도나한우', '북경(北京)', '명문식당', '일성스시', '새싹비빔밥']
[{'전현호': 3, '정성욱': 1, '박선녕': 11, '신상용': 0, '이정선': 7, '이희영': 3, '최미연': 2, '이재영': 11, '김재형': 0}, {'전현호': 6, '정성욱': 0, '박선녕': 0, '신상용': 0, '이정선': 1, '이희영': 0, '최미연': 2, '이재영': 1, '김재형': 2}, {'전현호': 3, '정성욱': 4, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 2, '이재영': 2, '김재형': 3}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 1, '이재영': 0, '김재형': 0}, {'전현호': 1, '정성욱': 2, '박선녕': 2, '신상용': 1, '이정선': 1, '이희영': 2, '최미연': 1, '이재영': 1, '김재형': 3}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 0, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 0, '박선녕': 1, '신상용': 0, '이정선': 1, '이희영': 1, '최미연': 0, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 6, '박선녕': 5, '신상용': 0, '이정선': 4, '이희영': 4, '최미연': 6, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 2, '이재영': 0, '김재형': 1}, {'전현호': 6, '정성욱': 4, '박선녕': 0, '신상용': 1, '이정선': 0, '이희영': 0,

## 동영상 파일 QR처리 끝

## Pandas 데이터프레임에 최종 입력 시작

In [210]:
import pandas as pd

column_list = ["직원성명", "노란코끼리", "어장촌생선구이", "횡성목장", "항도(港都)", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "구내매점"]
df = pd.DataFrame(columns=column_list)
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점


In [211]:
df['직원성명'] = name_list
for col_name, val_dict in zip(final_MA_list, final_MS_list):
	df[col_name] = val_dict.values()
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
0,전현호,3,6,3,0,1,0,0,0,0,6,0,1,NaN
1,정성욱,1,0,4,1,2,1,0,6,1,4,0,0,NaN
2,박선녕,11,0,0,0,2,0,1,5,0,0,1,0,NaN
3,신상용,0,0,0,0,1,0,0,0,0,1,0,0,NaN
4,이정선,7,1,0,0,1,0,1,4,0,0,1,0,NaN
5,이희영,3,0,0,0,2,0,1,4,0,0,3,0,NaN
6,최미연,2,2,2,1,1,0,0,6,2,1,0,0,NaN
7,이재영,11,1,2,0,1,0,0,0,0,5,0,0,NaN
8,김재형,0,2,3,0,3,0,0,0,1,5,0,0,NaN


In [212]:
df['구내매점'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
0,전현호,3,6,3,0,1,0,0,0,0,6,0,1,0
1,정성욱,1,0,4,1,2,1,0,6,1,4,0,0,0
2,박선녕,11,0,0,0,2,0,1,5,0,0,1,0,0
3,신상용,0,0,0,0,1,0,0,0,0,1,0,0,0
4,이정선,7,1,0,0,1,0,1,4,0,0,1,0,0
5,이희영,3,0,0,0,2,0,1,4,0,0,3,0,0
6,최미연,2,2,2,1,1,0,0,6,2,1,0,0,0
7,이재영,11,1,2,0,1,0,0,0,0,5,0,0,0
8,김재형,0,2,3,0,3,0,0,0,1,5,0,0,0


In [215]:
df2 = df.set_index('직원성명')
df2

,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
직원성명,,,,,,,,,,,,,
전현호,3,6,3,0,1,0,0,0,0,6,0,1,0
정성욱,1,0,4,1,2,1,0,6,1,4,0,0,0
박선녕,11,0,0,0,2,0,1,5,0,0,1,0,0
신상용,0,0,0,0,1,0,0,0,0,1,0,0,0
이정선,7,1,0,0,1,0,1,4,0,0,1,0,0
이희영,3,0,0,0,2,0,1,4,0,0,3,0,0
최미연,2,2,2,1,1,0,0,6,2,1,0,0,0
이재영,11,1,2,0,1,0,0,0,0,5,0,0,0
김재형,0,2,3,0,3,0,0,0,1,5,0,0,0


In [225]:
df3 = df2.T
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형
노란코끼리,3,1,11,0,7,3,2,11,0
어장촌생선구이,6,0,0,0,1,0,2,1,2
횡성목장,3,4,0,0,0,0,2,2,3
항도(港都),0,1,0,0,0,0,1,0,0
전주콩나루,1,2,2,1,1,2,1,1,3
옛날칼국수,0,1,0,0,0,0,0,0,0
칠리사이공,0,0,1,0,1,1,0,0,0
도나한우,0,6,5,0,4,4,6,0,0
북경(北京),0,1,0,0,0,0,2,0,1
명문식당,6,4,0,1,0,0,1,5,5


In [226]:
df3.loc['<식권 총계>'] = df3.sum(axis=0)
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형
노란코끼리,3,1,11,0,7,3,2,11,0
어장촌생선구이,6,0,0,0,1,0,2,1,2
횡성목장,3,4,0,0,0,0,2,2,3
항도(港都),0,1,0,0,0,0,1,0,0
전주콩나루,1,2,2,1,1,2,1,1,3
옛날칼국수,0,1,0,0,0,0,0,0,0
칠리사이공,0,0,1,0,1,1,0,0,0
도나한우,0,6,5,0,4,4,6,0,0
북경(北京),0,1,0,0,0,0,2,0,1
명문식당,6,4,0,1,0,0,1,5,5


In [227]:
df3['<식당별 식권 개수>'] = df3.sum(axis=1)
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형,<식당별 식권 개수>
노란코끼리,3,1,11,0,7,3,2,11,0,38
어장촌생선구이,6,0,0,0,1,0,2,1,2,12
횡성목장,3,4,0,0,0,0,2,2,3,14
항도(港都),0,1,0,0,0,0,1,0,0,2
전주콩나루,1,2,2,1,1,2,1,1,3,14
옛날칼국수,0,1,0,0,0,0,0,0,0,1
칠리사이공,0,0,1,0,1,1,0,0,0,3
도나한우,0,6,5,0,4,4,6,0,0,25
북경(北京),0,1,0,0,0,0,2,0,1,4
명문식당,6,4,0,1,0,0,1,5,5,22


In [228]:
df3.to_excel("12월 식권 사용내역.xlsx")

## 식권 처리 끝 !!